In [33]:
# create a Node class for characters
class Node():
    # initialize properties - characters, frequencies, children and code
    def __init__(self, char, val):
        self.char = char
        self.val = val
        self.left = None
        self.right = None
        self.code = ''
    # determine what property to compare 2 nodes on
    def __lt__(self, other):
        return self.val < other.val
    # to print out the nodes
    def __str__(self):
        return str(self.char) + ': ' + str(self.val)
    
# to maintain a min-heap structure, top to bottom
def heapify(node_list, i):
    # get the index of 2 children
    left = 2*i + 1
    right = 2*i + 2
    # compared with its children, get the smaller one
    smallest = i
    if left < len(node_list) and node_list[left] < node_list[i]:
        smallest = left
    if right < len(node_list) and node_list[right] < node_list[smallest]:
        smallest = right
    # if smaller than one of its children, swap
    if smallest != i:
        node_list[smallest], node_list[i] = node_list[i], node_list[smallest]
        # recursively fix the tree to maintain min-heap structure
        heapify(node_list, smallest)
        
# to maintain a min-heap structure, similar to the above, but traversing bottom to top
def heapify_2(node_list, i):
    bigger = i
    # get the parent
    if i % 2 == 0:
        p = int((i-2)/2)
    else:
        p = int((i-1)/2)
    if p >= 0 and node_list[i] < node_list[p]:
        bigger = p
    # if the parent is bigger, swap
    if bigger != i:
        node_list[i], node_list[bigger] = node_list[bigger], node_list[i]
        heapify_2(nodelist, bigger)
        
# build a min heap
def build_heap(node_list):
    for i in range(len(node_list)//2,-1,-1):
        heapify(node_list, i)

# get the smallest (root) of the heap
def heappop(node_list):
    # swap the root and the last leaf
    node_list[0], node_list[-1] = node_list[-1], node_list[0]
    # get the root
    small = node_list.pop()
    # maintain heap structure
    heapify(node_list, 0)
    return small

# insert an element to the heap
def heappush(node_list, new):
    node_list.append(new)
    heapify_2(node_list, len(node_list)-1)


# get the frequency of each distinct character
def freq_dict(text, freq_dict):
    # convert all to lowercase
    text = text.lower()
    for char in text:
        # add new key to the dictionary
        if char not in freq_dict:
            freq_dict[char] = 1
        # increase the frequency
        else:
            freq_dict[char] += 1
    return freq_dict

# build the code tree
def code_tree(freq_dict):
    # make a list of nodes of characters and their frequencies
    q = [Node(char = key, val = freq_dict[key]) for key in freq_dict]
    n = len(q)
    # make a min heap of frequencies
    build_heap(q)
    # build the code tree
    for i in range(n-1):
        z = Node(None, None)
        # remove the 2 smallest nodes and make them children of a new node
        z.left = x = heappop(q)
        z.right = y = heappop(q)
        z.val = x.val + y.val
        # push the new node to the heap
        heappush(q, z)
    return q

# encode each character
def encode(z, code_dict):
    # traverse down to the left
    if z.left != None:
        # add 0 to code if it's a left child
        z.left.code = z.code + '0'
        # if a code has a character, add to the code dictionary
        if z.left.char != None:
            code_dict[z.left.char] = z.left.code
        # recursively run down to the left
        encode(z.left, code_dict)
    # traverse down to the right
    if z.right != None:
        # add 1 to code if it's a right child
        z.right.code = z.code + '1'
        # if a code has a character, add to the code dictionary
        if z.right.char != None:
            code_dict[z.right.char] = z.right.code
        # recursively run down to the right
        encode(z.right, code_dict)
    return code_dict

# encode the text
def text_to_code(text, code_dict):
    # convert all to lowercase
    text = text.lower()
    code = code_dict
    code_string = ''
    # convert each character to its code and add to the code string
    for each in text:
        code_string += code[each]
    return code, code_string

# decode the text
def code_to_text(code_str, code_dict, new_text):
    # reverse key-value in the code dictionary
    reverse_code = {code_char: char for char, code_char in code_dict.items()}
    string = ''
    for i in code_str:
        string += i
        # if a code substring is valid
        if string in reverse_code:
            # add the corresponding character to the text
            new_text += reverse_code[string]
            string = ''
    return new_text

In [34]:
# use bitwise XOR to see if 2 substrings of the same length are similar
def compare(a, b):
    a = '0b' + a
    b = '0b' + b
    # get the XOR result
    c = bin(int(a, 2) ^ int(b, 2))
    d = len(a) - len(c)
    # make the result equal length to the string (in case of 0 at the beginning)
    if d > 0:
        c = '0b' + d*'0' + c[2:]
    return c

# see how similar 2 strings are by running one along the other and comparing
def similar(str1, str2, j, f, l):
    sim_list = {}
    # make the smaller one the first string - the one that runs
    if len(str1) > len(str2):
        str1, str2 = str2, str1
    k = -j
    # compare the end of first string and the start of the second
    while k >= -len(str1):
        a = str1[k:]
        b = str2[:-k]
        xor = compare(a, b)[2:]
        # see if there's any part that is very similar (the XOR result as a long string of 0s)
        if '0'*f in xor:
            sim_list[xor] = [a, b]
        # move the first string along the second one by j slots, until it is entirely compared to the starting part of the second part
        if k == -len(str1):
            break
        if k >= -len(str1) + j:
            k -= j
        else:
            k = -len(str1)
            
    s = j
    e = j + len(str1)
    # continue to move the first along to compare it with part of the 2nd one, until it reaches the end of the second one
    while e <= (len(str2)):
        a = str1
        b = str2[s:e]
        xor = compare(a, b)[2:]
        # see if there's any part that is very similar (the XOR result as a long string of 0s)
        if '0'*f in xor:
            sim_list[xor] = [a, b]
        if e == len(str2):
            break
        # move the first string along the 2nd one by j slots
        if e <= len(str2)-j:
            e += j
            s += j
        else:
            e = len(str2)
            s = e - len(str1)
            
    m = len(str1)-j
    # continue to move the first string along to compare the beginning of it to the end of the 2nd string, until it totally passes all of the 2nd one
    while m > 0:
        a = str1[:m]
        b = str2[-len(a):]
        xor = compare(a, b)[2:]
        # see if there's any part that is very similar (the XOR result as a long string of 0s)
        if '0'*f in xor:
            sim_list[xor] = [a, b]
        m = m-j
        
    # if there's more than 5 pairs of substrings that are similar, conclude that the 2 strings are similar
    if len(sim_list) > l:
        return len(sim_list), '2 sequences are somewhat similar'
    else:
        return len(sim_list), '2 sequences are not similar'


In [35]:
str_1 = 'CCTTGACCAATGACTTTCAAGTACCACGGAAAACAGGGGGGCAGAACTTCAGCAGTAAAGAATAAAAGGCCATGCAGAGAAGCAGCTGCACATGATCTGCTTCCGACACAGCTACAATCACCAGCGAGCTCTCAGACCTGACATCATGGTGCATTTTACTGCTGAGGAGAAGGCTGCCATCACTAGCCTGTGGGGCAAGATGAATGTGGAAGAGGCTGGAGGTGAAGCCTTGGGCAGGTAAGCAGTGGTTCTCAATGCATTGGGAATAAAGGGTGAGATATTACCTTTGCAAGTTGATTGGGAAAGTCTTCAAGATTTGTTAGCATCTCTAATGTTGTATCTGATATGGTGCCATTTCATAGGCTCCTGGTTGTTTACCCCTGGACCCAGAGATTTTTTGACAACTTTGGAAACCTGTCCTCTCCCCTGCCATCCTGGGCAACCCCAAGGTCAAGGCCCATGGCAAGAAGGTGCTGACATCCTTTGGAGATGCTATTAAAAACATGGATCAACCTCAAACCACCTTTGCTAAGCTAAGTGAGCTGCACTGTGACAAGTTGCATGTGGATCCTGAGAACTTCAGGGTGAGTTCAGGCGCGGGTGATGTGATTGTTTTGGCTTTCTTATTGACATTAATTGAGGTTGAGAATCTTATTGGAAACACCAGCAAAGATCTCAGAAATCATGGGTCTAGCTTGATTTTAGAACAGCAGACTTCGTAGTGTGCATAACCAAGGCTACCTTTGATTCAGAGCTAGTGACAGTAAAGGGCTACTTGGCTTAACTTTTAAGAAATCTTGCCAGAACTTGATGTGTTTATCCTGGAGAATAGTATTATAAAATTGTAGACTTGTGCAAGGAGAATGAAATTTGGCTTTTGATAGATGAAAGCCTGTTTCAAGGAAATAGAAATGCCTTATTTATGGGTCATGATAACTGAGGTTTAGGAAGAGATGTTTGAAACAAAAATTAAAAGATTTTCTCAAAGAAAAATAAGACTAATTTTCTAAAATAGATTAAATTTCCCATCAGTATTGTGACCAAGTGAAGGTTTGTTCTGTATTTGTTAGGGATTTTAAACCTCCATGAGAACTCTTGCAGCACTAACATTCTAAGTTTACAGAAATTAGATAACTGGTTAAAGAAAAATACTGGTAACATGAGGAGAGGGTGAGGGTATAGGTAGGCAGAATGTTGAATGTAAGGCTCATAAAATAAATTTGAACTCAAGCTCATCTGAACTTTTTGGTAGGCACAACACCTTGAAACAGTTTGAGGTCCAGGTTGCAAGGAATGTAGGTATAAAGCCTTTTTTTTTTTTTTTCAGCAAATTGTTTTTGAAAACTTCTGTTCAACATGCCTATGTGTTATTTTGTCTTTTGCCTAACAGCTCCTGGGTAACGTGATGGTGATTATTCTGGCTACTCACTTTGGCAGAGGAGTTCACTCCTGAGGTGCAGGCTGCCTGGCAGAAGCTGGTGTCGCTGTGGCCATTGCCTTGGGTCACAAGTACCACTGAGTTATCTCCCAGTTTGCCAGTGTTCCTGTGACCCTGACACCCTTCTTCTGCACATGAATACTGGGCTTGGCCTTGAGAGGAAGGTTTCTGTTTAATAAAGTACATTTTCTTCAGTAATCAAAATTGCAACTTCACTTCTGCCATCTTGTACTCTTGTGCTAAAGGAAAAG'

In [44]:
text_1 = code_to_text(coded_text_1[1], code_dictionary, '')
text_2 = code_to_text(coded_text_2[1], code_dictionary, '')
print(text_1.upper() == str_1, text_2.upper() == str_2)

True True


In [37]:
str_2 = 'CCTTGACCAATGACTTTTAAGTACCACGGAAAACAGGGGGGCAGAAGTTCAGCAGTAAAGAATAAAAGGCCATGCAGAGAAGCAGCTGCACATATCTGCTTCTGACACAGTACAATCACCAGCAAGCTCTCAGACCTGACATCATGGTGCATTTTACTGCTGAGGAGAAGGCATGCCATCACTAGCCTGTGGGGCAAGATGAATGTGGAAGAGGCTGGAGGTGAAGCCTTGGGCAGGTAAGCATTGGTTCTCAATGCATGGGAATAAAGGGTGAATATTACCTTTGCAAGCTGATTGGGAAAGTCTTCAAGATTTGATAGCATCTCTAATGTTGTATCTGATATGGTGCCATTTGCATAGGCTCCTGGTTGTTTACCCCTGGACCAGAGATTTTTCGACAACTTTGGAAACCTGTCCTCTCCCTCTGCCATCCTGGGCAACCCCAAGGTCAAGGCCCATGCAAGAAGGTGCTGACATCCTTTGGAGATGCTATTAAAAACATGGACAACCTCAAGACCACCTTTGCTAACTAATGAGCTGCACTGTGACAAGTTGCATGTGGATCCTGAGAACTTCAGGGTGAGTTCAGGCGCGGGTGATGTGATTTTTTTGGCTTTCTATATTGACATTAATTGAGGTTGAGAATCTTATTGGAAACACCAACAAAGATCTCAGAAATCATGGGTCTAGCTTGATTTTAGAACAGCAGACTTCTAGTGTGCATAACCAAGGCTAAACTTGATTCAGAGCTAGTGACAGTAAAGGGCTACTTGGCTTAACTTTTCAAGAAATCTTGCCAGAACTTGATGTGTTTATCCTGAGAATAGTATTAAAAATTGTAGACTTGTGCAAGGAGAATGAAATTTGGCTTTTGATAGATGAAAGCCTGTTTCAAGGAAATAGAAATGCCTTATTTATGGGTCATGATAACTGAGGTTTAGGAAGAGAGTTTGAACAAAAATTAAAAGATTTTTCTCAAAGAAAAATAAGACTAATTTTCTAAAATAGATTAAATTTCCCATCAGTATTGTGACCAAGTGAAGGTTTGTTTCTGTATTTGTTAAGGATTTTAAACCTTCATGAGAACTCTTGCAGCACTAACATTCTAAGTTTACAGAAATTAGATAACTGGTTAAAGAAAAATACTGGTAACATGAGGAGAGGGTGAGGGTATAGGTAGGTAGAATGTTGGAATGTAGGGCTCATAAAAATAAATTTGAACTCAAGCTCTATCTGAACTTTTTGGGTAGGCACAAACCTTGAAACAGTTTGAGGTCCAGGTTGTCAAGGAATGTAGGTATAAAGCCTTTTTTTTTTTTTTTCAGCAAATTGTTTTTGGAAACTTCTGTTCAACATGCCTATGTGTTATTTTGTCTTTTGCCTAACAGCTCCTGGGTAACGTGATGGTGATTATTCTGGCTACTCACTTTGGCAAGGAGTTCACTCCTGAGGTGCAGGCTGCCTGGCAGAAGCTGGTGTCTGCTGTGGCCATTGCCTTGGGTCACAAGTACCACTGAGTTATCTCCCAGTTTGCCAGTGTTCCTGTGACCCTGACACCCTTCTTCTGCACATGAATACTGGGCTTGGCCTTGAGAGGAAGGTTTCTGTTTAATAAAGTACATTTTCTTCAGTAATCAAAAATTTCAACTTTATCTTCTCCATCTTGTACTCTTGTGTTAAAGGAAAAG'

In [38]:
frequency_dict = freq_dict(str_1, {})
print(frequency_dict)

{'c': 309, 't': 503, 'g': 394, 'a': 483}


In [39]:
binary_code_tree = code_tree(frequency_dict)
print(binary_code_tree[0])

None: 1689


In [40]:
code_dictionary = encode(binary_code_tree[0], {})
print(code_dictionary)

{'c': '00', 'g': '01', 'a': '10', 't': '11'}


In [41]:
coded_text_1 = text_to_code(str_1, code_dictionary)
print(coded_text_1)

({'c': '00', 'g': '01', 'a': '10', 't': '11'}, '0000111101100000101011011000111111001010011110000010000101101010100010010101010101001001101000111100100100100111101010011010111010101001010000101101001001100110100100100100110100100010110110110011010011110000011000100010010011100010101100100000100100011001001100110010011000001101100010110010110101110100101111111110001101001101100101100110100101001101000010110010001110010000110111010101010010100110110110101101110101101001100101001101011001011101101001000011110101010010010111101001001001110101111100110010101101001011110101011010111010100101011101100110111011111000001111110100101001111101101111010101101010011100111100101001101111110111111001001011001100111010110111110111101100110110111011010111010000101111110010111001010011000011010111110111111110000000001101011000000010011001101111111111110110001010001111110101101010000011011100001100110000000011010000101100001101010100101000000000101001011100101001010000001011010100101001101001011101001101

In [42]:
# encode the 2nd string using the code of the 1st one
coded_text_2 = text_to_code(str_2, code_dictionary)
print(coded_text_2)

({'c': '00', 'g': '01', 'a': '10', 't': '11'}, '0000111101100000101011011000111111111010011110000010000101101010100010010101010101001001101001111100100100100111101010011010111010101001010000101101001001100110100100100100110100100010111011001101001111001101100010001001111000101011001000001001001010010011001100100110000011011000101100101101011101001011111111100011010011011001011001101001010010110100001011001000111001000011011101010101001010011011011010110111010110100110010100110101100101110110100100001111010101001001011110100100101111010111110011001010110100101101010110101110101001010111011010111011111000001111110100101001001101101111010101101010011100111100101001101111110110111001001011001100111010110111110111101100110110111011010111010000101111110100101110010100110000110101111101111111100000000011010110000010011001101111111111000110001010001111110101101010000011011100001100110000001100110100001011000011010101001010000000001010010111001010010100000010110100101001101001011101001101100010

In [43]:
result = similar(coded_text_1[1], coded_text_2[1], 5, 15, 5)
print(result)

(54, '2 sequences are somewhat similar')
